<a href="https://colab.research.google.com/github/PikasXYZ/OpenAI-convenient-chatBOT/blob/main/ChatGPT_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports & Functions

In [1]:
#Install & Clone
!pip install -q openai
!pip install -q python-docx
!git clone https://github.com/PikasXYZ/OpenAI-convenient-chatBOT.git #Credit to PikasXYZ

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Cloning into 'OpenAI-convenient-chatBOT'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 102 (delta 14), reused 0 (delta 0), pack-reused 73
Receiving objects: 100% (102/102), 595.83 KiB | 10.64 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [2]:
#Import
import json
import openai
import os
import gspread
import csv
import pandas as pd
import re
from oauth2client.service_account import ServiceAccountCredentials as SAC
from IPython.display import Markdown, display, HTML, Code, clear_output
from docx import Document

In [3]:
def get_text_before_first_symbol(input_string):
  for i, char in enumerate(input_string):
    if not char.isalnum() and char != ' ': return input_string[:i].strip()
  else: return input_string.strip()

get_text_before_first_symbol("  我喜歡妳？別開玩笑了XD")

'我喜歡妳'

In [4]:
def LoadHistory(history_path):
  doc = Document(history_path)
  messages = []
  for paragraph in doc.paragraphs:
    content = paragraph.text
    if content[:5]=="【You】": messages.append({"role": "user", "content": content.split("】")[1]})
    elif content[:9]=="【ChatGPT】": messages.append({"role": "assistant", "content": content.split("】")[1]})
    elif content[:8]=="【System】": messages.append({"role": "system", "content": content.split("】")[1]})
  else:
    return messages

LoadHistory("/content/OpenAI-convenient-chatBOT/history example.docx")

[{'role': 'system', 'content': ' You are a helpful assistant.'},
 {'role': 'user', 'content': ' This is a document for test'},
 {'role': 'assistant',
  'content': ' \n\nIf this text is read….\n\nIt means that there’s no problem of the function “LoadHistory”\n\n【Ok^^'}]

In [33]:
#Turn the Long Code of "Loading History""Deciding Whether to Use Prepared Prompt for Initialize" into a Function
def ChatInitialize(En):
  #Check if there's any history
  history_dir = "/content/drive/MyDrive/Colab Notebooks/ChatGPT_API"
  if not os.path.exists(history_dir): os.mkdir(history_dir)
  histories = [hd.replace(".docx","") for hd in os.listdir(history_dir) if ".docx" in hd]

  #Confirm whether user wants to load history if there's at least a history
  if histories:

    lst2show = histories.copy()
    id=0
    no = ""
    while("Loading History"):

      clear_output(wait = True)
      print("【History】" if En else "【聊天紀錄】")
      if lst2show:
        for i in range(id, id+5 if id+5<len(lst2show) else len(lst2show)):
          print("No.{}\t{}".format(i+1, lst2show[i]))
        print(f"page {1 + id//5}/{1 + len(lst2show)//5 if len(lst2show)%5 else len(lst2show)//5}\n" if En else f"頁數 {1 + id//5}/{1 + len(lst2show)//5 if len(lst2show)%5 else len(lst2show)//5}\n")

      else: 
        input(f"\"{no}\" not found! (Press Enter to reset)" if En else f"找不到含有「{no}」的聊天紀錄! (按Enter以重設)")
        lst2show = histories.copy()
        continue

      no = input("Enter the number of the history you want to load (\"N\"/\"P\" for next/previous page or \"Q\" to start new chat): " if En else "輸入紀錄的編號以匯入 (\"N\"/\"P\":上一頁/下一頁 或 \"Q\":開啟新話題): ")
      if no:

        try:
          if 0 <= int(no)-1 < len(lst2show):
            return LoadHistory(os.path.join(history_dir, lst2show[int(no)-1])+".docx"), 0, lst2show[int(no)-1]

        except:
          if no.lower() in ["n","p","q"]:
            if no.lower() == "n": id += 5 if id+5 < len(lst2show) else 0
            elif no.lower() == "p": id -= 5 if id-5 >= 0 else 0
            else: break
          else: lst2show, id = [h for h in histories if no in h], 0

      else: lst2show, id = histories.copy(), 0

  #Check user's intention to start with prompts prepared
  clear_output(wait = True)
  emphasis = 1
  while("Checking intention"):
    print("Wanna start with prompts prepared? {0}Y{0} or {0}N{0} ".format("\"" * emphasis) if En else "要使用既有的實用ChatGPT指令嗎? {0}Y{1}(要) 或 {0}N{1}(不要): ".format("「" * emphasis, "」" * emphasis))
    intention = input()
    if intention.lower() not in ["y","n"]: emphasis += 1
    else: break

  if intention.lower()=="n":
    return [{"role":"system", "content": "You are a helpful assistant." if En else "你是個得力助手"}], 0, ""
  else:
    #Prepare quick start list
    awesome_chatgpt_prompts = []
    if not En:
      with open("/content/OpenAI-convenient-chatBOT/Useful ChatGPT Prompts CN.csv","r") as f:
        useful_ChatGPT_prompts = [[row[0], "".join(row[1:])] for row in csv.reader(f)] #row[0] is act and row[1] is prompt
    else:
      with open("/content/OpenAI-convenient-chatBOT/Useful ChatGPT Prompts EN.csv","r") as f:
        useful_ChatGPT_prompts = [[row[0], "".join(row[1:])] for row in csv.reader(f)] #row[0] is act and row[1] is prompt
      
      !git clone https://github.com/f/awesome-chatgpt-prompts.git #Credit to Fatih Kadir Akın
      with open("/content/awesome-chatgpt-prompts/prompts.csv","r") as f:
        awesome_chatgpt_prompts = [[row[0], "".join(row[1:])] for row in csv.reader(f) if row[0] != "act"] #row[0] is act and row[1] is prompt

    quick_start = useful_ChatGPT_prompts + awesome_chatgpt_prompts

    #quick start!
    id=0
    lst2show = quick_start.copy()
    no = ""
    first_content = ""
    while(not first_content):

      clear_output(wait = True)
      if En:
        print("【Quick Start List】" if lst2show == quick_start.copy() else f"【Search Results of {no}】")
      else:
        print("【快速開始清單】" if lst2show == quick_start.copy() else f"【「{no}」的搜尋結果】")
      if lst2show:
        for i in range(id, id+5 if id+5<len(lst2show) else len(lst2show)):
          print(f"No.{i+1}\t{lst2show[i][0]}")
        print("page" if En else "頁數" ,f" {1 + id//5}/{1 + len(lst2show)//5 if len(lst2show)%5 else len(lst2show)//5}\n")

      else: 
        input(f"\"{no}\" not found! (Press Enter to reset)" if En else f"找不到含有「{no}」的指令! (按Enter以重設)")
        lst2show = quick_start.copy()
        continue

      no = input("Enter the number of the mode you want (\"N\"/\"P\" for next/previous page or \"Q\" to quit): " if En else "輸入想要的指令號碼 (\"N\"/\"P\":上一頁/下一頁 或 \"Q\":不想用現成指令): ")
      if no:

        try:
          if 0 <= int(no)-1 < len(lst2show):
            act = lst2show[int(no)-1][0]
            print(f"\n【{act}】")
            system_content = "You are a "+act if act in awesome_chatgpt_prompts else ("Please "+act if En else "請幫我"+act)
            first_content = lst2show[int(no)-1][1]
            if "{" in first_content:
              print(">>>", first_content, "\nThere\'re some parameters to set..." if En else "\n有參數須設定...")
              for keyword in [x for i, x in enumerate(re.findall(r'\{([^\{\}]+)\}', first_content)) if x not in re.findall(r'\{([^\{\}]+)\}', first_content)[:i]]:
                globals()[keyword.replace(" ","_")] = input(keyword + " = ")
                first_content = first_content.replace("{"+keyword+"}",globals()[keyword.replace(" ","_")]) 
        except:
          if no.lower() in ["n","p","q"]:
            if no.lower() == "n": id += 5 if id+5 < len(lst2show) else 0
            elif no.lower() == "p": id -= 5 if id-5 >= 0 else 0
            else: return [{"role":"system", "content": "You are a helpful assistant." if En else "你是個得力助手"}], 0, ""
          else: lst2show, id = [row for row in quick_start if no in row[0] or no in row[1]], 0

      else: lst2show, id = quick_start.copy(), 0

    else:
      messages = [{"role":"system", "content": system_content}]
      messages.append({"role": "user", "content": first_content})
      chatCompletion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
      reply = chatCompletion['choices'][0]['message']['content']
      messages.append({"role": "assistant", "content": reply})

    return messages, chatCompletion['usage']['total_tokens'], ""

In [6]:
#Chat & Save Chat Record
def ChatNSave(messages, token_spent, En, history_name):
  clear_output(wait = True)
  #display(Markdown("Chat Start! (Enter \"Q\" merely if you wanna finish the chat.)"))
  print("Chat Start! (Enter \"Q\" merely if you wanna finish the chat.)" if En else "可以開始聊天了! (只輸入 \"Q\" 就能結束聊天)")

  doc = Document()
  for elem in messages:
    if elem["role"] != "system":
      content = "【{}】 ".format("You" if elem["role"] == "user" else "ChatGPT") + elem["content"]
      #display(Markdown(content))
      print("\n" + content)
      doc.add_paragraph(content)
    else: doc.add_paragraph("【System】 "+ elem["content"])

  try:
    message = input("\n【You】 ")
    while message.lower() != "q":
      messages.append({"role": "user", "content": message})
      doc.add_paragraph('【You】 '+message)
      chatCompletion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
      token_spent += chatCompletion['usage']['total_tokens']
      reply = chatCompletion['choices'][0]['message']['content']
      messages.append({"role": "assistant", "content": reply})
      doc.add_paragraph('【ChatGPT】 '+reply)
      #display(Markdown(f"【ChatGPT】 {reply}"))
      print(f"\n【ChatGPT】 {reply}")
      message = input("\n【You】 ")
    else:
      print(f"\n\nChat Finished. ({token_spent} tokens were spent this time.)" if En else f"\n\n聊天已結束. (這次共花費 {token_spent} 個 token)")
      raise
  except:
    emphasis = 1
    while("Checking intension to save record"):
      willing = input("\nWanna save the chat record? {0}Y{0} or {0}N{0}: ".format("\"" * emphasis) if En else "\n要將聊天紀錄儲存起來嗎? {0}Y(是){1} or {0}N(否){1}:".format("「" * emphasis, "」" * emphasis))
      if willing.lower() not in ["y","n"]: emphasis += 1
      else: break
    if willing.lower() == "y": 
      chatCompletion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages = messages+[{"role": "user", "content": "I wanna save all we chatted as a docx, name it within 5 words. give me the name only, no other words and no Filename Extension" if En else "濃縮本次聊天的內容於十個字以內回傳(無標點符號)，不要輸出其他內容"}])
      reply = chatCompletion['choices'][0]['message']['content']
      docx_name = history_name if history_name else get_text_before_first_symbol(re.sub(r'[^\w\s]', '', reply))
      doc.save(os.path.join("/content/drive/MyDrive/Colab Notebooks/ChatGPT_API",f"{docx_name}.docx"))
      print("Result was saved to" if En else "聊天紀錄儲存至", os.path.join("/content/drive/MyDrive/Colab Notebooks/ChatGPT_API",f"{docx_name}.docx"))
    else:
      print("Result wasn't saved." if En else "聊天紀錄未儲存><")

#ChatBOT

In [28]:
#@title Language Setting
#@markdown ##### 請設定欲使用的語言 (英文 : EN / 中文 : CN)
#@markdown ##### Please set the language you wanna use. (English : EN / Chinese : CN)
#@markdown ##### ㅤ
language = "EN" #@param {type:"string"}
if language.upper() not in ["EN","CN"]:
  raise Exception("無法辨識的語言，請將 language 設定為 EN 或 CN\nInvalid language, please key in  EN or CN after \"language:\" above.")
En = language == "EN"

In [29]:
#@title Preprocessing
#@markdown #####若不想從雲端匯入 ChatGPT 的 API 金鑰，請輸入一個確定能用的
#@markdown #####If you don't want to import API keys from Google Drive, Please key in an usable one.
#@markdown ##### ㅤ
API_key = "在此輸入 API 金鑰 / Key in API key here" #@param {type:"string"}

api_keys = []
if API_key[:2] == "sk":
  api_keys.append(API_key)

else:
  #Link to Google Drive
  from google.colab import drive
  drive.mount('/content/drive')
  clear_output()

  #Find "ChatGPT_API_keys.txt" to load API keys from user's Google Drive
  for root, dirs, files in os.walk("/content/drive/MyDrive/"):
    if "ChatGPT_API_keys.txt" in files: 
      txt_path = os.path.join(root, "ChatGPT_API_keys.txt")
      break
  else:
    print("Cannot find \"ChatGPT_API_keys.txt\" in your Google Drive." if En else "在您的 Google Drive 裡找不到 \"ChatGPT_API_keys.txt\"")

  try:
    with open(txt_path,"r") as f:
      api_keys = [rl.strip() for rl in f.readlines()] #an API key per line
    print("Found ChatGPT API keys in ChatGPT_API_keys.txt!\n" if En else "已在 ChatGPT_API_keys.txt 中找到 ChatGPT API 的金鑰!\n")

  except:
    for root, dirs, files in os.walk("/content/drive/MyDrive/"): 
      if "GSheet.json" in files: 
        GSheet_path = os.path.join(root, "GSheet.json") #key of Google Sheet API
        break
    else:
      raise Exception("Cannot find \"GSheet.json\" (json of Google Sheet API key)." if En else "找不到 \"GSheet.json\" (Google Sheet API 金鑰的 json 檔)")

    #Get Google Sheets API Authorization
    print("\nInstalling neccesaries..." if En else "\n正在安裝必要的套件...")
    !pip install -q gspread 
    !pip install -q oauth2client
    #@markdown ##### ㅤ　　
    #@markdown ##### ㅤ
    #@markdown #####若您的雲端中有名為 `GSheet.json` 的 Google Sheet API 金鑰檔案且已將 ChatGPT API 的金鑰存在某個已串接 Google Sheet API 的 Google Sheet 裡，可以在底下輸入該 Google Sheet 的 id 以便匯入
    #@markdown #####If there's a Google Sheet API file named `GSheet.json` in your Google Drive and your ChatGPT API keys are stored in a Google Sheet which already connects to Google Sheet API, you can key in the id of that Google Sheet for searching.
    #@markdown ##### ㅤ
    sheet_id = "在此輸入id / Key in id here." #@param {type:"string"}
    scopes = ["https://spreadsheets.google.com/feeds"]
    credentials = SAC.from_json_keyfile_name(GSheet_path, scopes) #key location
    client = gspread.authorize(credentials)
    Sheet = client.open_by_key("sheet_id") #The longest garbled character in the URL of Google Sheet
    sheet = Sheet.sheet1 
    api_keys = [row[0] for row in sheet.get_all_values()] #an API key per row at column A
    print("Found ChatGPT API keys in your Google Sheet!\n" if En else "已在您的 Google Sheet 中找到可用的 ChatGPT API 金鑰!\n")

#Find an Usable API Key
for api_key in api_keys:
  try:
    openai.api_key = api_key
    chatCompletion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": "hello"}])
    print("Found an usable API key!" if En else "找到可用的API金鑰!",f" ({api_key[:13]}...)")
    break
  except Exception as e:
    print(type(e).__name__," : ",str(e),f" ({api_key[:13]}...)")
else:
  raise Exception("There\'s no usable API key, please try something new!" if En else "沒有可用的API金鑰，請再次確認您的資料!")

已在 ChatGPT_API_keys.txt 中找到 ChatGPT API 的金鑰!

RateLimitError  :  You exceeded your current quota, please check your plan and billing details.  (sk-MCqZ0udF3Q...)
RateLimitError  :  You exceeded your current quota, please check your plan and billing details.  (sk-PCe8WHbyUj...)
找到可用的API金鑰!  (sk-bwk0Vt0aGE...)


In [32]:
#@title Start Chatting!
#Initialize Chat & Start Chating
messages, token_spent, history_name = ChatInitialize(En)
ChatNSave(messages, token_spent, En, history_name)

可以開始聊天了! (只輸入 "Q" 就能結束聊天)

【You】 Q


聊天已結束. (這次共花費 0 個 token)

要將聊天紀錄儲存起來嗎? 「Y(是)」 or 「N(否)」:N
聊天紀錄未儲存><
